In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-


# Computer vision
## Session 13b
### Basic Operation CV2

<img src='../../prasami_images/prasami_color_tutorials_small.png' width='400' alt="By Pramod Sharma : pramod.sharma@prasami.com"/>

In [ ]:
# Import some libraries
import os

import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

import cv2

%matplotlib inline

In [ ]:
# Some basic parameters
inpDir = '../../input'
outDir = '../output'
dataDir = 'basic_operations'

RANDOM_STATE = 24

np.random.seed(RANDOM_STATE) # Set Random Seed for reproducible  results

# parameters for Matplotlib
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 12),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'
         }

CMAP = 'jet'

LINE_THICK = 1

MARK_COLOR = (15, 82, 186)

plt.rcParams.update(params)

## Helper Functions

In [ ]:
def fn_read_image(fileName):
    '''
    Args:
        fileName : Path of image file to read
    returns:
        im: image in cv2 format
        rgbIm: image in RGB format
    
    '''
    imgPath = os.path.join(inpDir, dataDir, fileName)
    
    # Read image file
    im = cv2.imread(imgPath, cv2.IMREAD_COLOR)
    
    
    if im is None:
        print('Could not open or find the image:', fileName)
        exit(0)
    else:
        # convert to RGB image
        rgbIm = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
        return im, rgbIm

    
def fn_plot_one_img(im):
    
    '''
    Args:
        im : image to display and save
    
    '''
    # showing image
    plt.imshow(im)
    plt.axis('off')
    plt.savefig(os.path.join(outDir, imgFileName))
        
def fn_plot_images(im_lst):
    '''
    Args:
        img_list: list of images
    '''
    nRows = 1
    nCols = len(im_lst)
    
    
    fig, axes = plt.subplots(nRows, nCols)
    
    for i in range(nCols):
        axes[i].imshow(im_lst[i]['img'], cmap = im_lst[i]['cmap'])
        axes[i].set_title(im_lst[i]['name'])

        axes[i].set_xticklabels([]);
        axes[i].set_yticklabels([]);

                
    plt.tight_layout()

## Low Contrast Images

### Histogram Equalization

Histogram $\rightarrow$ how many pixel with each of value between 0 and 255.

<img src='../../images/Histograms.png' width='600' alt="Histogram" />


Plotting Histogram of image

In [ ]:
# load bright image
bFileName = '/home/hpcap/Desktop/modules/Ai/Data/input-20240618T113733Z-001/input/basic_operations/11.jpeg' #'DSC_2477.JPG'

brightImg, brightRBG = fn_read_image(bFileName)

# Resizing to reduce processing time
#brightImg = cv2.resize(brightImg, ( 1024, 688))
#brightRBG = cv2.resize(brightRBG, ( 1024, 688))


# load dark image
dFileName = '/home/hpcap/Desktop/modules/Ai/Data/input-20240618T113733Z-001/input/basic_operations/094.JPG'#'kids.jpeg'

darkImg, darkRBG  = fn_read_image(dFileName)

# Resizing to reduce processing time
#darkImg = cv2.resize(darkImg, ( 1024, 688))
#darkRBG = cv2.resize(darkRBG, ( 1024, 688))


img_lst = [{'img': brightRBG, 'name': 'Bright Image','cmap' : CMAP},
           {'img': darkRBG, 'name': 'Dark Image','cmap' : CMAP}]

fn_plot_images(img_lst)

In [ ]:
darkImg.shape

In [ ]:
plt.figure(figsize=(15,6))
plt.hist(brightImg.flatten(), bins = 25, alpha = 0.7, 
         color = 'orange', ec = 'r')

plt.hist(darkImg.flatten(), bins = 25, alpha = 0.7, 
         color = 'lightblue', 
         ec = 'b')

plt.grid()

plt.show()

In [ ]:
# Calculate histogram using cv2.calcHist()
brightHist = cv2.calcHist([brightImg], [0], None, [256], [0,256])
darkHist = cv2.calcHist([darkImg], [0], None, [256], [0,256])

# Display the histogram
plt.figure(figsize=(15,6))
plt.plot(brightHist)
plt.plot(darkHist)
plt.grid();

### 2 D histogram

We have seen 1 D histograms so far. We need to use 2 D histograms, if we want to find correlation between two channels, say Blue and Green. Function is same. Some of the parameters are as follows:
- Channels: [ 0, 1 ], [ 1, 2 ], [ 0, 2 ] for (Blue, Green),  (Green, Red) and (Blue, Red) respectively.
- bins: for each channel separately, say [256, 256]
- range: [0,256, 0, 256] for 8-bit image


In [ ]:
# first convert to HSV. In HSV mode 2 channels can represent color. 
brightHSV = cv2.cvtColor(brightImg, cv2.COLOR_BGR2HSV)
darkHSV = cv2.cvtColor(darkImg, cv2.COLOR_BGR2HSV)

# 2D histogram for Blue and Green channels.
brightHist = cv2.calcHist([brightHSV], [0, 1], None, [100, 100], 
                          [0, 256, 0, 256])

darkHist = cv2.calcHist([darkHSV], [0, 1], None, [100, 100], 
                        [0, 256, 0, 256])


# show using matplotlib
fig, axes = plt.subplots(1, 2)

ax = axes[0]
ax.imshow(brightHist, interpolation='nearest', cmap = 'gray')
ax.set_xticklabels([]);
ax.set_yticklabels([]);

ax = axes[1]
ax.imshow(darkHist, interpolation='nearest', cmap = 'gray')
ax.set_xticklabels([]);
ax.set_yticklabels([]);
plt.tight_layout()

### Histogram Equalization
Equalizes the histogram of a grayscale image.

Parameters
- src	Source 8-bit single channel image.
- dst	Destination image of the same size and type as src .


In [ ]:
equalBlue = cv2.equalizeHist(darkImg[:, :, 0]) 
equalGreen = cv2.equalizeHist(darkImg[:, :, 1]) 
equalRed = cv2.equalizeHist(darkImg[:, :, 2]) 

equalRBG = np.zeros_like(darkImg)
equalRBG[:, :, 0] = equalRed
equalRBG[:, :, 1] = equalGreen
equalRBG[:, :, 2] = equalBlue

img_lst = [{'img': darkRBG, 'name': 'Original RGB','cmap' : CMAP},
           {'img': equalRBG, 'name': 'Equalized','cmap' : CMAP}]

fn_plot_images(img_lst)

cv2.imwrite(os.path.join(outDir,'dark_image_equal.png'), 
            cv2.cvtColor(equalRBG, cv2.COLOR_RGB2BGR))

### Adaptive Histogram Equalization

What if image has significant dark portions.

In [ ]:
# Resizing to reduce processing time
#darkImg = cv2.resize(darkImg, ( 1024, 688))

#darkRBG = cv2.cvtColor(darkImg, cv2.COLOR_BGR2RGB)

# create a CLAHE object (Arguments are optional).
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
claheBlue = clahe.apply(darkImg[:,:, 0])
claheGreen = clahe.apply(darkImg[:,:, 1])
claheRed = clahe.apply(darkImg[:,:, 2])

claheImg = np.zeros_like(darkImg)

claheImg [ :, :, 0] = claheRed
claheImg [ :, :, 1] = claheGreen
claheImg [ :, :, 2] = claheBlue

 
img_lst = [{'img': darkRBG, 'name': 'Original RGB','cmap' : CMAP},
           {'img': claheImg, 'name': 'Equalized','cmap' : CMAP}]

fn_plot_images(img_lst)


cv2.imwrite(os.path.join(outDir,'dark_image_clahe.png'), cv2.cvtColor(claheImg, cv2.COLOR_RGB2BGR))

## Interpolations

A number of interpolation techniques are available as shown below:
<img src='../../images/Interpolation.png' width='600' alt="Interpolation Techniques" />

Non-adoptive perform interpolation in a fixed pattern for every pixel, while adoptive algorithms detect local spacial features, such as edges, of the pixel neighborhood and make effective choices.

### Nearest Neighbor Interpolation

$$\begin{bmatrix} 10 & 20 \\ 30 & 40\end{bmatrix}\stackrel{2 x}{\rightarrow}\begin{bmatrix} 10 & 10 & 20 & 20 \\10 & 10 & 20 & 20 \\ 30 & 30 & 40 & 40\\ 30 & 30 & 40 & 40\end{bmatrix}$$

### Bi-linear Interpolation

Linear interpolation in both the directions.

- Fix corner pixel as per the original image and then interpolate all intermediate values.

$$\begin{bmatrix} 10 & 20 \\ 30 & 40\end{bmatrix}\stackrel{2 x}{\rightarrow}\begin{bmatrix} 10 & 12 & 17 & 20 \\15 & 17 & 22 & 25 \\ 25 & 27 & 32 & 35\\ 30 & 32 & 37 & 40\end{bmatrix}$$

- This  performs better than nearst neighbor but at sharp edges, this algorithm is not ideal.

### Bi-cubic Interpolation
- Bi-linear uses 4 nearest neighbors to get the output while Bi-cubic uses 16 (4 x 4) neighbors.
- Weight distribution too is little different

$
\begin{aligned}
fx & = ( dx + 0.5 ) * scale_x - 0.5\\
sx & = cvFloor(fx)\\
fx & -= sx\\
\end{aligned}
$

**Note:** cvFloor roundoff to lower int



$
\begin{aligned}
A & = -0.75f\\
coeffs[0] & = ((A*(x + 1) - 5*A)*(x + 1) + 8*A)*(x + 1) - 4*A\\
coeffs[1] & = ((A + 2)*x - (A + 3))*x*x + 1\\
coeffs[2] & = ((A + 2)*(1 - x) - (A + 3))*(1 - x)*(1 - x) + 1\\
coeffs[3] & = 1.f - coeffs[0] - coeffs[1] - coeffs[2]\\
\end{aligned}
$

$$\begin{bmatrix} 10 & 20 \\ 30 & 40\end{bmatrix}\stackrel{2 x}{\rightarrow}\begin{bmatrix} 7 & 10 & 16 & 19 \\13 & 17 & 22 & 26 \\ 24 & 28 & 33 & 37\\ 31 & 34 & 40 & 43\end{bmatrix}$$

* Notably sharper image than previous two and balances processing time and output quality. Hence, widely used. 


## Interpolation

In [ ]:
# load bright image
imgFileName = '/home/hpcap/Desktop/modules/Ai/Data/input-20240618T113733Z-001/input/basic_operations/car_2_small.png'
oFileName = '/home/hpcap/Desktop/modules/Ai/Data/input-20240618T113733Z-001/input/basic_operations/car_2_cropped.png'

srcImg, rgbImg = fn_read_image(imgFileName)
oImg, oRgbImg  = fn_read_image(oFileName)

img_lst = [{'img': oImg, 'name': 'Original Image','cmap' : CMAP},
           {'img': oRgbImg, 'name': 'Original RGB Image','cmap' : CMAP}]

fn_plot_images(img_lst)

img_lst = [{'img': srcImg, 'name': 'Reduced Source Image','cmap' : CMAP},
           {'img': rgbImg, 'name': 'Reduced rgb Image','cmap' : CMAP}]

fn_plot_images(img_lst)



### Nearest

In [ ]:
nearImg = cv2.resize(rgbImg, None, fx = 10, fy = 10, 
                     interpolation = cv2.INTER_NEAREST)

img_lst = [{'img': rgbImg, 'name': 'rgb Image','cmap' : CMAP},
           {'img': nearImg, 'name': 'Nearest Neighbor','cmap' : CMAP}]
print ('Shape of source', rgbImg.shape, 'Shape of destination', nearImg.shape)
fn_plot_images(img_lst)

This produces a pixlated image, no new data!

## Bi-Linear

In [ ]:
bLinImg = cv2.resize(rgbImg, None, fx = 10, fy = 10, 
                     interpolation = cv2.INTER_LINEAR)

img_lst = [{'img': rgbImg, 'name': 'rgb Image','cmap' : CMAP},
           {'img': bLinImg, 'name': 'Bi-Linear','cmap' : CMAP}]

fn_plot_images(img_lst)

Looks smooth, but edges are blurred!

### Bi-Cubic Interpolation

In [ ]:
bCubImg = cv2.resize(rgbImg, None, fx = 10, fy = 10,
                     interpolation = cv2.INTER_CUBIC)

img_lst = [{'img': rgbImg, 'name': 'rgb Image','cmap' : CMAP},
           {'img': bCubImg, 'name': 'Bi-Cubic','cmap' : CMAP}]

fn_plot_images(img_lst)

In [ ]:
img_lst = [oRgbImg, nearImg,bLinImg, bCubImg]


stackedImg = np.hstack(img_lst)

# Display the images
fn_plot_one_img(stackedImg)

### RGB Track Bar

In [ ]:
def nothing(x):
    '''
     Do nothing ! Good!
    '''
    pass
 
# Create a black image, a window
blankImg = np.zeros((512,512,3), np.uint8)

cv2.namedWindow('image',cv2.WINDOW_NORMAL)

# create trackbars for color change
cv2.createTrackbar('Red','image',0,255,nothing)
cv2.createTrackbar('Green','image',0,255,nothing)
cv2.createTrackbar('Blue','image',0,255,nothing)


while True:
    
    cv2.imshow('image',blankImg)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    # get current positions of three trackbars
    r = cv2.getTrackbarPos('Red','image')
    g = cv2.getTrackbarPos('Green','image')
    b = cv2.getTrackbarPos('Blue','image')
    
    print (r, b, g)
    
    blankImg[:] = [b,g,r]
    
cv2.destroyAllWindows()

In [ ]:
# Create a black image, a window
blankImg = np.zeros((512,512,3), np.uint8)
cv2.namedWindow('TrackerBars',cv2.WINDOW_NORMAL)

# create trackbars for color change
cv2.createTrackbar('H','TrackerBars',0,180,nothing)
cv2.createTrackbar('S','TrackerBars',0,255,nothing)
cv2.createTrackbar('I','TrackerBars',0,255,nothing)

while(True):
    cv2.imshow('image',blankImg)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    # get current positions of four trackbars
    h = cv2.getTrackbarPos('H','TrackerBars')
    s = cv2.getTrackbarPos('S','TrackerBars')
    v = cv2.getTrackbarPos('I','TrackerBars')
    
    blankImg[:,:] = [h,s,v]
    blankImg = cv2.cvtColor(blankImg, cv2.COLOR_HSV2BGR)
    
cv2.destroyAllWindows()

## Arithmetic Operations

In [ ]:
# Load original image
imgFileName = '/home/hpcap/Desktop/modules/Ai/Data/input-20240618T113733Z-001/input/basic_operations/094.JPG'

srcImg, rgbImg = fn_read_image(imgFileName)

# Resizing to fit terminal
srcImg = cv2.resize(srcImg, ( 1280, 960))

# Create list to store noisy images
images = []

# Generate noisy images using cv2.randn. Can use your own mean and std.
for _ in range(20):
    
    img1 = srcImg.copy() 
    
    cv2.randn(img1,(0,0,0),(50,50,50))
    
    images.append ( srcImg + img1 )

    
# For averaging create an empty array, then add images to this array.

avgImg = np.zeros((srcImg.shape[0],
                   srcImg.shape[1],
                   srcImg.shape[2]), np.float32)

for im in images:
    
    avgImg = avgImg + im / 20


# Round the float values. Always specify the dtype

avgImg = np.array ( np.round ( avgImg ), dtype=np.uint8 )

rgbImg = cv2.cvtColor(srcImg, cv2.COLOR_BGR2RGB)
avgImg = cv2.cvtColor(avgImg, cv2.COLOR_BGR2RGB)
noisyImg = cv2.cvtColor(images[0], cv2.COLOR_BGR2RGB)

# Display the images
img_lst = [{'img': rgbImg, 'name': 'Original Image','cmap' : CMAP},
           {'img': noisyImg, 'name': 'Noisy','cmap' : CMAP},
           {'img': avgImg, 'name': 'Averaged Image','cmap' : CMAP}]

fn_plot_images(img_lst)

In [ ]:
fn_plot_one_img(avgImg)

## Intensity Transformation

Basic types of transformation functions used for image enhancement are:
- Linear (Negative and Identity Transformations)
- Log and inverse-log transformations
- Power Law transformations

### Negative Transformation

In [ ]:
# Load original image
imgFileName = '/home/hpcap/Pictures/trial.jpeg'

srcImg, rgbImg = fn_read_image(imgFileName)

# Create Negative image
negImg = cv2.bitwise_not(srcImg)

#rgbImg = cv2.cvtColor(srcImg, cv2.COLOR_BGR2RGB)
negImg = cv2.cvtColor(negImg, cv2.COLOR_BGR2RGB)

# Display the images
img_lst = [{'img': rgbImg, 'name': 'Original Image','cmap' : CMAP},
           {'img': negImg, 'name': 'Negative','cmap' : CMAP}]

fn_plot_images(img_lst)

### Log Transformation

In [ ]:
# Apply log transformation method
c = 255 / np.log1p(1+np.max(srcImg))


logImg = np.array((c * (np.log1p(1+srcImg))), dtype = 'uint8')

rgbImg = cv2.cvtColor(srcImg, cv2.COLOR_BGR2RGB)
logImg = cv2.cvtColor(logImg, cv2.COLOR_BGR2RGB)

# Display the images
img_lst = [{'img': rgbImg, 'name': 'Original Image','cmap' : CMAP},
           {'img': logImg, 'name': 'Log Transformation','cmap' : CMAP}]

fn_plot_images(img_lst)

### Gamma Transformation

In [ ]:
file_list = []

# Trying 4 gamma values.
for gamma in [0.1, 0.5, 1.2, 2.2]:
      
    # Apply gamma correction.
    gamma_corrected = np.array(255*(srcImg / 255) ** gamma, 
                               dtype = 'uint8')
  
    # create gamma file name
    gmaFileName = os.path.join(outDir, 
                               f'{imgFileName}_gamma_{gamma}.jpg')
    
    # Save edited images.
    cv2.imwrite(gmaFileName,
                gamma_corrected)
    
    file_list.append(gmaFileName)
    
img_list = []

for f in  file_list:
    
    gImg = cv2.imread(f, cv2.IMREAD_COLOR)
    
    gImg = cv2.cvtColor(gImg, cv2.COLOR_BGR2RGB)
    
    img_list.append(gImg)

gammaImg = np.hstack(img_list)

# Display the images
fn_plot_one_img(gammaImg)

In [ ]:
# Load original image
imgFileName = '/home/hpcap/Desktop/modules/Ai/Data/input-20240618T113733Z-001/input/basic_operations/bright_image.jpeg'

def fn_empty(a):
    pass

# Create a window
cv2.namedWindow('TrackerBars',cv2.WINDOW_NORMAL)

# create trackbars for color change
cv2.createTrackbar('Hue Min', 'TrackerBars',   0, 179, fn_empty)
cv2.createTrackbar('Hue Max', 'TrackerBars',  19, 179, fn_empty)
cv2.createTrackbar('Sat Min', 'TrackerBars', 110, 255, fn_empty)
cv2.createTrackbar('Sat Max', 'TrackerBars', 240, 255, fn_empty)
cv2.createTrackbar('Val Min', 'TrackerBars', 153, 255, fn_empty)
cv2.createTrackbar('Val Max', 'TrackerBars', 250, 255, fn_empty)

while(True):
    
    srcImg, rgbImg = fn_read_image(imgFileName)
    #srcImg = cv2.resize(srcImg, ( 960, 1280))
    
    hsvImg = cv2.cvtColor(srcImg, cv2.COLOR_BGR2HSV)
                
    # get current positions of four trackbars
    h_min = cv2.getTrackbarPos('Hue Min','TrackerBars')
    h_max = cv2.getTrackbarPos('Hue Max','TrackerBars')
    s_min = cv2.getTrackbarPos('Sat Min','TrackerBars')
    s_max = cv2.getTrackbarPos('Sat Max','TrackerBars')
    v_min = cv2.getTrackbarPos('Val Min','TrackerBars')
    v_max = cv2.getTrackbarPos('Val Max','TrackerBars')
     
    lower = np.array([h_min, s_min, v_min])
    
    upper = np.array([h_max, s_max, v_max])
    
    maskImg = cv2.inRange(hsvImg, lower, upper)

    cv2.imshow('Original',srcImg)
    cv2.imshow('HSV',hsvImg)
    cv2.imshow('Mask',maskImg)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()